In [1]:
# addprocs(30);
addprocs(7);

In [2]:
require(:POMDPModels)
require(:POMCP)

In [3]:
@everywhere begin
    using POMDPModels
    using POMCP
    using POMDPs
    using POMDPToolbox
    import POMCP.init_V
    import POMDPs: action, updater
end

In [4]:
# N = 100000;
N = 100;
problem = BabyPOMDP(-5, -10);

In [5]:
function est_reward(problem, policy, belief, N; eps=0.01)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        sim = POMDPToolbox.RolloutSimulator(rng=sim_rng, initial_state=false, eps=eps)
        up = updater(policy)
        POMDPs.simulate(sim, problem, policy, up, initialize_belief(up,belief))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [6]:
@time est_reward(problem, FeedWhenCrying(), false, N)

  1.123545 seconds (468.53 k allocations: 19.794 MB, 0.92% gc time)


-15.061634270536405

This is better than in the crying babies test because epsilon is large and, more importantly, it gets a notcrying observation on the first step every time

In [7]:
# Random
pol_rng = MersenneTwister(7)
@time est_reward(problem, RandomPolicy(problem, rng=pol_rng), BoolDistribution(0.5), N)

  0.780705 seconds (493.48 k allocations: 19.884 MB, 0.59% gc time)


-31.33487686696136

In [8]:
# POMCP with FWC rollout policy
rng = MersenneTwister(3)

solver = POMCPSolver(rollout_solver=FeedWhenCrying(),
                    eps=0.01,
                    c=10.0,
                    tree_queries=300, 
                    rng=rng,
                    node_belief_updater=updater(problem))
                    

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

 15.883813 seconds (196.03 k allocations: 8.463 MB)


-14.946426206647613

In [9]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(rollout_solver=RandomPolicy(problem, rng=rollout_pol_rng),
                     eps=0.01,
                     c=10.0,
                     tree_queries=300, 
                     rng=rng,
                     node_belief_updater=updater(problem))

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

 15.864060 seconds (15.32 k allocations: 1.229 MB)


-15.943953697101506

In [10]:
# Optimal policy for these particular problem parameters:
# if the belief that the baby is hungry is over .28206, then feed (see DMU book)
@everywhere begin
    type OptBabyPolicy <: POMDPs.Policy end
    function action(p::OptBabyPolicy, b::BoolDistribution, a=false)
        a = b.p>0.28206
        return a
    end
    updater(::OptBabyPolicy) = updater(BabyPOMDP(-5,-10))
end
@time est_reward(problem, OptBabyPolicy(), BoolDistribution(0.0), N)

  0.225786 seconds (49.64 k allocations: 2.445 MB)


-14.686851194997828

In [11]:
# POMCPDPW with FWC rollout policy
rng = MersenneTwister(2)

solver = POMCPDPWSolver(rollout_solver=FeedWhenCrying(),
                    eps=0.01,
                    c=10.0,
                    tree_queries=300, 
                    rng=rng,
                    node_belief_updater=updater(problem))
                    

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

 16.407194 seconds (230.56 k allocations: 9.915 MB)


-14.898076931340603